In [1]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install requests
import requests
import hashlib
import json
from binascii import unhexlify, hexlify

In [3]:
def readBlock(hash, printouts=False):
    # api-endpoint
    # URL = "https://blockchain.info/rawblock/000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f?format=hex"
    # URL = "https://blockchain.info/rawblock/00000000000000001e8d6829a8a21adc5d38d0a473b144b6765798e61f98bd1d?format=hex"
    URL = "https://blockchain.info/rawblock/" + hash + "?format=hex"

    # sending get request and saving the response as response object
    r = requests.get(url = URL) #, params = PARAMS)
    
    # print(r.content)
    if printouts: print('text: ' + r.text)
    if printouts: print('encoding: ' + r.encoding)
    # print('type: ' + type(r.text))

    blockHex = r.text
    # head = r.text[:size2]
    global curr
    curr = 0

    def read(size):
        global curr
        prev = curr
        curr += size * 2
    #     if size == 1:
    #         print("prev: ", prev)
    #         print("curr: ", curr)
    #         print("head sect: ", blockHex[prev:curr])
        return blockHex[prev:curr]

    def readInt(size=4):
        tempArr = bytearray(bytes.fromhex(read(size)))
    #     if size == 1:
    #         print("tempArr: ", tempArr)
        return int.from_bytes(tempArr, "little")

    def readHash(size=32):
        tempArr = bytearray(bytes.fromhex(read(32)))
        tempArr.reverse()
        return tempArr

    def readVarInt():
        size = readInt(1)
    #     print("readvarint size: ", size)
        if size < 253:
            return size
        elif size == 253:
            return readInt(2)
        elif size == 254:
            return readInt(4)
        elif size == 255:
            return readInt(8)
        return -1

    def readTxInput():
        tx_input = {}
        tx_input["prev_out"] = readHash().hex()
        tx_input["txOutId"] = readInt(4)
        tx_input["scriptLen"] = readVarInt()
        tx_input["script"] = read(tx_input["scriptLen"])
        tx_input["sequence"] = readInt(4)
        return tx_input

    def readTxInputs(vin_sz):
        outputs = []
        for i in range(0, vin_sz):
            outputs.append(readTxInput())
        return outputs
        
    def readTxOutput():
        tx_output = {}
        tx_output["value"] = readInt(8)
        tx_output["scriptLen"] = readVarInt()
        tx_output["pubkey"] = read(tx_output["scriptLen"])
        return tx_output
        
    def readTxOutputs(vout_sz):
        outputs = []
        for i in range(0, vout_sz):
            outputs.append(readTxOutput())
        return outputs
        
    def readTx(n_tx):
        outputs = []
        for i in range(0, n_tx):
            tx = {}
            tx["ver"] = readInt()
            tx["vin_sz"] = readVarInt()
            tx["inputs"] = readTxInputs(tx["vin_sz"])
            tx["vout_sz"] = readVarInt()
            tx["outputs"] = readTxOutputs(tx["vout_sz"])
            tx["lock_time"] = readInt()
            outputs.append(tx)
        return outputs

    returnObj = {}

    if printouts: print("===HEADER===")

    returnObj["ver"] = readInt()
    if printouts: print("ver: ", returnObj["ver"])

    returnObj["prev_block"] = readHash().hex()
    if printouts: print("prev_block: ", returnObj["prev_block"])

    returnObj["mrkl_root"] = readHash().hex()
    if printouts: print("mrkl_root: ", returnObj["mrkl_root"])

    returnObj["time"] = readInt()
    if printouts: print("time: ", returnObj["time"])
    
    URL2 = "https://min-api.cryptocompare.com/data/v2/histoday?fsym=BTC&tsym=USD&limit=1&aggregate=1&toTs=" + str(returnObj["time"])
    r2 = requests.get(url=URL2).json()
    print("r2: ", r2)
    returnObj["price"] = r2["Data"]["Data"][-1]["close"]
#     returnObj["price"] = json.loads(r2).get("close")

    returnObj["bits"] = readInt()
    if printouts: print("bits: ", returnObj["bits"])

    returnObj["nonce"] = readInt()
    if printouts: print("nonce: ", returnObj["nonce"])

    if printouts: print("===TRANSACTIONS===")

    returnObj["n_tx"] = readVarInt()
    if printouts: print("n_tx: ", returnObj["n_tx"])

    returnObj["tx"] = readTx(returnObj["n_tx"])
    if printouts: print("tx: ", returnObj["tx"])

    returnObjJson = json.dumps(returnObj)
    if printouts: print(json.dumps(returnObj, indent=2, sort_keys=False))

    return returnObjJson

# head2 = unhexlify(head)
# head3 = hashlib.sha256(hashlib.sha256(head2).digest()).digest()
# head4 = hexlify(head3[::-1]).decode("utf-8")
# print('head4: ' + head4)
# toBytes = bytes.fromhex(blockHex)
# print(toBytes)
# toAscii = toBytes.decode("ascii")#("ASCII")

# print(toAscii)

# body2 = unhexlify(body)
# body3 = hashlib.sha256(hashlib.sha256(body2).digest()).digest()
# body4 = hexlify(body3[::-1]).decode("utf-8")
# print('body4: ' + body4)

# b = bytes.fromhex(blockHex)
# # print(b)
# ba = bytearray(b)
# ba.reverse()
# # print(ba)
# a = ba.decode("ascii")

In [8]:
# readBlock("000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f", True)
readBlock("000000000000034a7dedef4a161fa058a2d67a173a90155f3a2fe6fc132e0ebf", False)

r2:  {'Response': 'Success', 'Message': '', 'HasWarning': False, 'Type': 100, 'RateLimit': {}, 'Data': {'Aggregated': False, 'TimeFrom': 1348185600, 'TimeTo': 1348272000, 'Data': [{'time': 1348185600, 'high': 12.5, 'low': 12.01, 'open': 12.28, 'volumefrom': 35834.78, 'volumeto': 439674.22, 'close': 12.37, 'conversionType': 'direct', 'conversionSymbol': ''}, {'time': 1348272000, 'high': 12.4, 'low': 12.14, 'open': 12.37, 'volumefrom': 14986.56, 'volumeto': 183186.34, 'close': 12.24, 'conversionType': 'direct', 'conversionSymbol': ''}]}}


'{"ver": 1, "prev_block": "000000000000017a86f33937e38281248ed65512f390bf175b60a50f321c85bf", "mrkl_root": "577ae98bea85808127d42b0a5e1232035e80568d7a16e0fc1cfa6dd81e611c41", "time": 1348309604, "price": 12.24, "bits": 436591499, "nonce": 887684620, "n_tx": 239, "tx": [{"ver": 1, "vin_sz": 1, "inputs": [{"prev_out": "0000000000000000000000000000000000000000000000000000000000000000", "txOutId": 4294967295, "scriptLen": 72, "script": "1265636f406f7a636f2e696e202f503253482f0464925d500214012cfabe6d6d3b0959d2d70774e7cff5b0e632f29312eb8596b6e5d24fcb65e8fc02c9b2a7bf0100000000000000", "sequence": 4294967295}], "vout_sz": 1, "outputs": [{"value": 5021355000, "scriptLen": 25, "pubkey": "76a9144cc96dd327f28d2a8787abf23498a7878037e1a388ac"}], "lock_time": 0}, {"ver": 1, "vin_sz": 2, "inputs": [{"prev_out": "7050b98c4215a00522ef90cfe13f45ddf2a4a199d348f377f56b0a47f5ee18e4", "txOutId": 1, "scriptLen": 107, "script": "483045022100b1e81a3fa44fa924c395b571f7799f5c98c4051029d8a143fa54343e61af867b0220377